In [1]:
import tensorflow as tf

In [2]:
import pathlib

In [3]:
data_root = pathlib.Path('./data/flower_photos')

In [4]:
for item in data_root.iterdir():
    print(item)

data/flower_photos/daisy
data/flower_photos/dandelion
data/flower_photos/LICENSE.txt
data/flower_photos/roses
data/flower_photos/sunflowers
data/flower_photos/tulips


#### Getting the path of all the images.

In [14]:
all_img_path = [str(x) for x in list(data_root.glob('*/*'))]

#### Getting the labels from the given directories

In [20]:
label_name = []
for i in data_root.glob('*/'):
    if i.is_dir():
        label_name.append(str(i.name))

In [22]:
label_name.sort()

In [24]:
label2idx = dict((name, idx) for idx, name in enumerate(label_name))

In [30]:
all_img_labels = []
for i in all_img_path:
    label = label2idx[pathlib.Path(i).parent.name]
    all_img_labels.append(label)

## tf.io.read_file

Returns a tensor of the given file. It is in string format. Maybe for uniformity.

In [33]:
img_raw = tf.io.read_file(all_img_path[9])

## tf.image.decode_image

Accepts a 0-D string as input. This has to be the raw image I presume.

In [64]:
img_tensor = tf.image.decode_image(img_raw)

## Reshaping tensor, 

`tf.image.resize(tnsr, [dim1, dim2])`

In [43]:
img_final = tf.image.resize(img_tensor, [192, 192])

## Put decode and resizing into a method

In [72]:
def process_image(image):
    img = tf.image.decode_jpeg(image)
    img = tf.image.resize(img, [192, 192])
    img /= 255.0
    return img

## Load and process image

In [70]:
def load_process_img(img_path):
    img_raw = tf.io.read_file(img_path)
    t_img = process_image(img_raw)
    return t_img

## Converting to a dataset

The way we can use datasets in tf, based on the one tutorial I read, is as follows.

1. Create a data set for the path to the files.
2. Create a new dataset that maps the path dataset with a method that reads and returns tensors.
3. Create a dataset that contains the labels. (Make sure the labels are tensors)
4. Zip the two datasets together to make a single dataset that returns tensor images, labels

In [54]:
path_ds = tf.data.Dataset.from_tensor_slices(all_img_path)

In [73]:
image_ds = path_ds.map(load_process_img)

In [74]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_img_labels, tf.int64))

In [76]:
dataset = tf.data.Dataset.zip((image_ds, label_ds))